## Without using Pipeling Training

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv("./DATA/titanic.csv")

In [3]:
df.sample()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
487,488,0,1,"Kent, Mr. Edward Austin",male,58.0,0,0,11771,29.7,B37,C


In [4]:
df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], inplace=True)

In [5]:
# Train, Test, Split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=["Survived"]),
    df["Survived"],
    test_size=0.2
)

In [6]:
X_train.head(5), y_train.head(5)

(     Pclass   Sex   Age  SibSp  Parch     Fare Embarked
 690       1  male  31.0      1      0  57.0000        S
 663       3  male  36.0      0      0   7.4958        S
 326       3  male  61.0      0      0   6.2375        S
 529       2  male  23.0      2      1  11.5000        S
 73        3  male  26.0      1      0  14.4542        C,
 690    1
 663    0
 326    0
 529    0
 73     0
 Name: Survived, dtype: int64)

In [7]:
X_train.value_counts().sum()

574

In [8]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         136
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [9]:
# Imputer

imputer_age = SimpleImputer()
imputer_embarked = SimpleImputer(strategy="most_frequent")

X_train_age = imputer_age.fit_transform(X_train[["Age"]])
X_train_embarked = imputer_embarked.fit_transform(X_train[["Embarked"]])

X_test_age = imputer_age.transform(X_test[["Age"]])
X_test_embarked = imputer_embarked.transform(X_test[["Embarked"]])

In [14]:
# OHE for categorical

ohe_sex = OneHotEncoder(handle_unknown="ignore")
ohe_embarked = OneHotEncoder(handle_unknown="ignore")

X_train_sex = ohe_sex.fit_transform(X_train[["Sex"]]).toarray()
X_train_embarked = ohe_embarked.fit_transform(X_train[["Embarked"]]).toarray()

X_test_sex = ohe_sex.transform(X_test[["Sex"]]).toarray()
X_test_embarked = ohe_embarked.transform(X_test[["Embarked"]]).toarray()

In [11]:
X_train_others = X_train.drop(columns=["Sex", "Age", "Embarked"])
X_test_others = X_test.drop(columns=["Sex", "Age", "Embarked"])

In [15]:
X_train_transformed = np.concatenate(
    (X_train_others, X_train_age, X_train_sex, X_train_embarked),
    axis=1
)

X_test_transformed = np.concatenate(
    (X_test_others, X_test_age, X_test_sex, X_test_embarked),
    axis=1
)

In [16]:
X_train_transformed[0]

array([ 1.,  1.,  0., 57., 31.,  0.,  1.,  0.,  0.,  1.,  0.])

In [17]:
X_train_transformed.shape, X_test_transformed.shape

((712, 11), (179, 11))

In [18]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train_transformed, y_train)

DecisionTreeClassifier()

In [19]:
y_pred = classifier.predict(X_test_transformed)

In [20]:
from sklearn.metrics import accuracy_score


accuracy_score(y_test, y_pred)

0.8268156424581006

In [21]:
import pickle

pickle.dump(ohe_sex, open("./temp/ohe_sex.pkl", "wb"))
pickle.dump(ohe_embarked, open("./temp/ohe_embarked.pkl", "wb"))
pickle.dump(classifier, open("./temp/classifier.pkl", "wb"))

In [24]:
ohe_sex_loading = pickle.load(open('./temp/ohe_sex.pkl', "rb"))
ohe_embarked_loading = pickle.load(open("./temp/ohe_embarked.pkl", 'rb'))
classifier_loading = pickle.load(open("./temp/classifier.pkl", "rb"))

In [25]:
test_input = np.array([2, "male", 31.0, 0, 0, 10.5, "S"], dtype=object).reshape(1, 7)
test_input

array([[2, 'male', 31.0, 0, 0, 10.5, 'S']], dtype=object)

In [29]:
test_input_sex = ohe_sex.transform(test_input[:, 1].reshape(1, 1)).toarray()
test_input_embarked = ohe_embarked.transform(test_input[:, -1].reshape(1, 1)).toarray()

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [30]:
test_input_sex, test_input_embarked

(array([[0., 1.]]), array([[0., 0., 1., 0.]]))

In [36]:
test_input_age = test_input[:, -2].reshape(1, 1)
test_input_age

array([[10.5]], dtype=object)

In [35]:
test_input_model = np.concatenate(
    (test_input[:, [0, 3, 4, 5]], test_input_age, test_input_sex, test_input_embarked),
    axis=1
)

In [34]:
test_input_model.shape

(1, 11)

In [37]:
classifier_loading.predict(test_input_model)

array([1], dtype=int64)

## Using Sklearn Pipeline approach

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [39]:
df = pd.read_csv("./DATA/titanic.csv")

In [40]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Impute missing values => OHE => Scale => Feature Selection => Model Training

In [41]:
df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], inplace=True)

In [42]:
# Train Test & Split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=["Survived"]),
    df["Survived"],
    test_size=0.2
)

In [43]:
X_train.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
466,2,male,NaN,0,0,0.0000,S
698,1,male,49.0,1,1,110.8833,C
735,3,male,28.5,0,0,16.1000,S
162,3,male,26.0,0,0,7.7750,S
115,3,male,21.0,0,0,7.9250,S


In [44]:
y_train.head(5)

466    0
698    0
735    0
162    0
115    0
Name: Survived, dtype: int64

In [ ]:
trf1 = ColumnTransformer([
    ("impute_age", SimpleImputer(), [2]), # index number 
    ("impute_embarked", SimpleImputer(strategy="most_frequent"), [6]) #column number 6
], remainder="passthrough")

In [61]:
trf2 = ColumnTransformer([
    ("ohe_sex_embarked", OneHotEncoder(handle_unknown="ignore", sparse_output=False), [1, 6])
], remainder="passthrough")

In [62]:
trf3 = ColumnTransformer([
    ("scale", MinMaxScaler(), slice(0,8))
])

In [63]:
trf4 = SelectKBest(score_func=chi2, k=5) # select best 5 feature

In [64]:
trf5 = DecisionTreeClassifier()

In [65]:
pipe = Pipeline([
    ("trf1", trf1),
    ("trf2", trf2),
    ("trf3", trf3),
    ("trf4", trf4),
    ("trf5", trf5),
])

In [66]:
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x000002A431A8F920>)),
                ('trf5', DecisionTreeClassifier())])

In [67]:
make_pipe = make_pipeline( # no need to give name to the pipeline
    trf1,
    trf2,
    trf3,
    trf3,
    trf4,
    trf5
)

In [68]:
make_pipe

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('columntransformer-4',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('selectkbest',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x000002A431A8F920>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [69]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x000002A431A8F920>)),
                ('trf5', DecisionTreeClassifier())])

In [70]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 8, None))]),
 'trf4': SelectKBest(k=5, score_func=<function chi2 at 0x000002A431A8F920>),
 'trf5': DecisionTreeClassifier()}

In [71]:
y_pred = pipe.predict(X_test)

In [72]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.6145251396648045

## Cross Validation

In [73]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe, X_train, y_train, cv=5, scoring="accuracy").mean()

0.6418693982074263

## GridSearch Cross Validation

In [81]:
params = {
    "trf5__max_depth": [1,2,3,4,5,None] # should give original name of the pipe here
}

In [82]:
from sklearn.model_selection import GridSearchCV

grid_cv = GridSearchCV(pipe, params, cv=5, scoring="accuracy")
grid_cv.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 8, None))])),
                                       ('trf4',
                                        SelectKBest(k=5,
                                                    score_func=<function chi2 at 0x000002A431A8F920>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [83]:
grid_cv.best_score_

0.6418693982074263

In [86]:
grid_cv.best_params_

{'trf5__max_depth': 1}

In [87]:
# saving model

import pickle

pickle.dump(pipe, open("./temp/pipe.pkl", "wb"))

In [92]:
loading_model = pickle.load(open("./temp/pipe.pkl", "rb"))
loading_model

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x000002A431A8F920>)),
                ('trf5', DecisionTreeClassifier())])

In [94]:
loading_model[-1]

DecisionTreeClassifier()